<h1><center>

![](https://i.imgur.com/jrRGMfx.png)

AI PRACTITIONER COURSE 

PRACTICE LAB - SESSION 16 - Sequential decision making: Tabular Q-learning & DQN 

**&copy; 2020 VTCA-COTAI. Internal Use Only.**

</center></h1>


# Giới thiệu Reinforcement learning (Rl) - learning to act

Reinforcement learning - học tăng cường được định nghĩa một cách tổng quan nhất là các bài toán trong đó mục tiêu là để tối đa hóa phần thưởng trong dài hạn. Ví dụ khi chơi cờ, điều mà người chơi có kinh nghiệm phải tính toán là khi đi mỗi nước cờ hoặc cụm các nước cờ bằng cách nào đấy đều phải đóng góp đến kết quả toàn cục.

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/c/cf/CaroVN.jpg" height="270">
</center>

Mô hình của DQN gồm *tác nhân (agent)* tương tác với *môi trường (environment)* tại *trạng thái (state)* thông qua các *hành động (action)* tại thời điểm $t$ và nhận về *phần thưởng (reward)*. Sau mỗi bước hành động tác nhân sẽ chuyển tiếp sang trạng thái mới. Một vòng lặp các phản ứng qua lại giữa tác nhân - môi trường được mô phỏng như hình sau:

<center>
<img src="https://miro.medium.com/max/904/1*WOYVzYnF-rbdcgZU2Wt9Yw.png" height="200">
</center>

Trong đó:

- Trạng thái thể hiện tình trạng của tác nhân và môi trường tại thời điểm $t$.

- Tập hợp bộ hành động tác nhân có thể làm tại mỗi trạng thái.

- Tập hợp bộ phần thưởng cho từng trạng thái - hành động.

***Tổng quan là:***

> *Với trạng thái hiện tại, bộ hành động và tập hợp các phần thưởng tương ứng đã được quy định trước, máy sẽ học cách chọn ra hành động để tối đa hóa phần thưởng nhận được trong tương lai.*

Vậy chắc chắn là ở đây đang tồn tại 1 cái hàm nào đó có đầu vào đầu ra. Đầu vào bao gồm các trạng thái, bộ action có thể thực hiện tại từng trạng thái, reward cho mỗi cặp trạng thái - hành động. Còn đầu ra là hành động nên làm.

Và cái hàm mà nhận vào 1 bảng (ta gọi là bảng *Q-value*) và trả ra hành động tương ứng gọi là *hàm chiến lược (policy)*, ký hiệu là $\pi$. 

***Như vậy:***

> *Thuật toán sẽ đi tìm hàm chiến lược tối ưu kia. Tối ưu dựa trên tiêu chí là, nó sẽ học cách tính toán hành động nào nên làm để tối đa phần thưởng nhiều nhất có thể trong tương lai.*







# Code với game Hồ băng

Mô tả: Game hồ băng, chia làm 16 ô vuông gồm ô bắt đầu, các ô vuông có băng hoặc không băng và ô đích. Ô bắt đầu chú thích là `S`, ô có băng chú thích là `F`, ô là một cái hố chú thích là `H`, ô đích đến chú thích là `G`. Nhiệm vụ của bạn là di chuyển từ `S` đến `G`. Nếu đi lọt vào ô hố thì game over!



![alt text](https://miro.medium.com/max/884/1*MCjDzR-wfMMkS0rPqXSmKw.png)
<center>


<img src="https://miro.medium.com/max/842/1*Qp14HWQfOeE2UoSxrxCxAg.png" height="270">
</center>



### Q-values table

|State(cell)|Left|Down|Right|Up|
|---|---|---|---|---|
|00 | 0 | 0 | 0 | 0 |
|01 | 0 | 0 | 0 | 0 |
|...| 0 | 0 | 0 | 0 |
|15 | 0 | 0 | 0 | 0 |

In [ ]:
import numpy as np
import os
import time
import gym
import random
from gym.envs.registration import register

## Load and setup the environment
env = gym.make('FrozenLake-v0', is_slippery=False)
#To start, we need to reset to the initial (default) state
env.reset()
#Get the total number of possibles states (obervations)
states = env.observation_space.n
#Get the total number of actions available to the agent
actions = env.action_space.n

In [ ]:
#To visualize the current state
env.render()


SFFF
FHFH
FFFH
HFFG


In [ ]:
## Initialize Q-table structure
#---->TODO<----: create a table to contain expected values at given states
Q = np.zeros(shape = (16,4))

In [ ]:
## For Q-learning
epsilon = 0.99 #For the epsilon-greedy approach
gamma = 0.9 #Discount factor
eta = 0.8 #Learning rate
#The maximum amount of times we'll run the game
total_episodes = 1000

#The maximum steps we'll run for every episode 
max_steps = 100

# Phân tích theo TEFPA

- Task: input là state, output là optimal action.

- E: 
  + Input: trạng thái hiện tại, hành động kế tiếp
  + Output: phần thưởng ở hiện tại.

- F: hàm chiến lược $\pi$. 

- P: Phương trình Bellman:

$$
Q^{target}(s_t,a_t) \approx r_t + \gamma max_{a_{t+1}}(Q(s_{t+1},a_{t+1}))
$$


$$
P = \Delta{Q} = \hat{Q}(s_t,a_t) - Q^{target}(s_t,a_t)
$$

- A:

$$
\hat{Q}(s_t,a_t) = \hat{Q}(s_t,a_t) - \eta*\Delta{Q} \\
= \hat{Q}(s_t,a_t) +\eta[r_t + \gamma max_{a_{t+1}}(Q(s_{t+1},a_{t+1})) - \hat{Q}(s_t,a_t)]
$$




In [ ]:
#---->TODO:
def choose_action(Q,state, epsilon):
    #Randomly generate a number between 0 and 1 and see if it's smaller than epsilon
    ep = np.random.uniform(0,1)
    #If smaller, a random action is chosen using env.action_space.sample()
    if ep < epsilon :
        return env.action_space.sample()
    #If greater, we choose the action having the maximum value in the Q-table for that state: a.k.a best action
    else :
        return np.argmax(Q[state])

#---->TODO: code Q-learning based on Q formula
#input: old_value: Q[s,a] 
#       max_value_of_next_state: max(a') Q[s',a'] 
#output: new_value: Q[s,a] updated
def update_Q(old_value, max_value_of_next_state, reward):
    return old_value + eta*(reward+gamma*max_value_of_next_state-old_value)


In [ ]:
## Start
rList = []
for episode in range(total_episodes):
    #Store the initial state using env.reset()
    state = env.reset()
    #Store the number of steps
    t = 0
    #Reduce epsilon gradually
    epsilon = max(0.01, epsilon-0.001)

    rAll = 0
    while t < max_steps:
        # env.render()
        #---->TODO: Choose appropriate action
        action = choose_action(Q,state,epsilon)
        #After the action taken in the environment, the reward for that action and next_state is returned
        #env.step() returns a tuple
        #done (bool type) returns True if the espisode is finished
        #info (dict type) stores the extra information for debugging purposes
        next_state, reward, done, info = env.step(action) 
        #---->TODO: Append reward into rList
        rList.append(reward)
        #Update Q-table 
        old_value = Q[state,action]
        max_value_of_next_state = max(Q[next_state])
        #---->TODO: Update Q-value
        Q[state,action] = update_Q(old_value,max_value_of_next_state,reward)

        #Set the current state as the next state   
        state = next_state
        t += 1

        if done:
            break
    rList.append(rAll)
print(Q)
    # print(Q, episode, epsilon)

[[0.531441 0.59049  0.59049  0.531441]
 [0.531441 0.       0.6561   0.59049 ]
 [0.59049  0.729    0.59049  0.6561  ]
 [0.6561   0.       0.59049  0.59049 ]
 [0.59049  0.6561   0.       0.531441]
 [0.       0.       0.       0.      ]
 [0.       0.81     0.       0.6561  ]
 [0.       0.       0.       0.      ]
 [0.6561   0.       0.729    0.59049 ]
 [0.6561   0.81     0.81     0.      ]
 [0.729    0.9      0.       0.729   ]
 [0.       0.       0.       0.      ]
 [0.       0.       0.       0.      ]
 [0.       0.81     0.9      0.729   ]
 [0.81     0.9      1.       0.81    ]
 [0.       0.       0.       0.      ]]


In [ ]:
## Visualize the movements

#---->TODO: get a list of best actions manually

#---->TODO: visualize
from IPython.display import clear_output
import time

state = env.reset()
env.render()
clear_output(wait=True)
time.sleep(1)
while True:
  action = np.argmax(Q[state])
  next_state, reward, done, _ = env.step(action)
  env.render()
  clear_output(wait=True)
  time.sleep(1)

  if done:
    break
  state = next_state



  (Right)
SFFF
FHFH
FFFH
HFFG
